# Calculate X-ray telescope coverage of LoVoCCS clusters

## Import Statements

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from astropy.units import Quantity, UnitConversionError
from astropy.cosmology import LambdaCDM
import matplotlib.pyplot as plt
from typing import Union, List
from shutil import rmtree
import os
import json
from matplotlib import pyplot as plt
from matplotlib.patches import Circle

import xga
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.products.phot import ExpMap
from xga.imagetools.misc import pix_deg_scale, physical_rad_to_pix
from xga.imagetools.profile import annular_mask
from xga.sourcetools.misc import rad_to_ang

# This is a bit cheeky, but suppresses the warnings that XGA spits out (they are 
#  useful, but not when I'm trying to present this notebook on GitHub)
import warnings
warnings.filterwarnings('ignore')

# Set up a variable that controls how long individual XSPEC fits are allowed to run
timeout = Quantity(6, 'hr')

%matplotlib inline

## Checking other repository is present

This notebook relies on the LoVoCCS coverage maps generated for the X-LoVoCCS-Data repository (which contains all the notebooks necessary to recreate the LoVoCCS X-ray dataset) - thus we need to ensure that the repository is where we expect it to be before this notebook continues (this is mostly in case anyone ever wishes to recreate all of our analyses):

In [2]:
if not os.path.exists("../../../X-LoVoCCS-Data/"):
    raise FileNotFoundError("We require the X-LoVoCCS-Data repository to be cloned into the same directory as "\
                            "the X-LoVoCCS-Analyses directory.")
else:
    cov_map_path = "../../../X-LoVoCCS-Data/outputs/coverage_maps/fits/{}"

## Define useful functions

Here we define any functions to be used in this notebook - for this particular analysis this means the function that can mask a coverage map and calculate how much the defined region has been covered by observations by different X-ray telescopes:

In [3]:
def cov_calc(rel_row, extra_apertures=None) -> dict:
    
    cov_path = os.path.join(cov_map_path, "{n}_550pix_20.0arcsec.fits".format(n=rel_row["LoVoCCS_name"]))
    cur_cheese = ExpMap(cov_path, '', '', '', '', '', Quantity(0.5, 'keV'), Quantity(2.0, 'keV'))
    
    with fits.open(cov_path) as covero:
        print(covero[0].header['MISSION*'])
            
        cov_arr = covero[0].data
#         for
        xmm_cov_arr = cov_arr[0, :, :]
        chandra_cov_arr = cov_arr[1, :, :]
        
        print(xmm_cov_arr.sum() / (xmm_cov_arr.shape[0]*xmm_cov_arr.shape[1]))
        stop
        
        field_chandra_cov_frac.append(chandra_cov_arr.sum() / (chandra_cov_arr.shape[0]*chandra_cov_arr.shape[1]))
        
        pix_cen = Quantity([xmm_cov_arr.shape[0]/2, xmm_cov_arr.shape[1]/2], 'pix').round(0).astype(int)
        rad = Quantity(row['R500'], 'Mpc')
        
        pix_rad = physical_rad_to_pix(cur_cheese, rad, pix_cen, row['redshift'], cosmo)
        
        rad_mask = annular_mask(pix_cen, np.array([0]), np.array([pix_rad.value]), xmm_cov_arr.shape)
        msk_xmm_cov = xmm_cov_arr*rad_mask
        r500_xmm_cov_frac.append(msk_xmm_cov.sum() / rad_mask.sum())
        
        msk_chandra_cov = chandra_cov_arr*rad_mask
        r500_chandra_cov_frac.append(msk_chandra_cov.sum() / rad_mask.sum())
        
        rel_samp.loc[row_ind, 'XMM_R500_FRAC'] = r500_xmm_cov_frac[row_ind]
        rel_samp.loc[row_ind, 'CHANDRA_R500_FRAC'] = r500_chandra_cov_frac[row_ind]

## Reading in the sample

We read in the LoVoCCS sample relevant to the current work:

In [6]:
samp = pd.read_csv("../../sample_files/X-LoVoCCSI.csv")
samp['LoVoCCS_name'] = samp['LoVoCCSID'].apply(lambda x: "LoVoCCS-"+str(x))
samp

,LoVoCCSID,Name,MCXC_RA,MCXC_DEC,MCXC_Redshift,manual_xray_ra,manual_xray_dec,LoVoCCS_name
0,1,A2029,227.73000,5.720000,0.0766,227.734300,5.745471,LoVoCCS-1
1,2,A401,44.74000,13.580000,0.0739,NaN,NaN,LoVoCCS-2
2,4,A85,10.45875,-9.301944,0.0555,NaN,NaN,LoVoCCS-4
3,5,A3667,303.13000,-56.830000,0.0556,303.157313,-56.845978,LoVoCCS-5
4,7,A3827,330.48000,-59.950000,0.0980,NaN,NaN,LoVoCCS-7
...,...,...,...,...,...,...,...,...
61,121,A3128,52.50000,-52.600000,0.0624,52.466189,-52.580728,LoVoCCS-121
62,122,A1023,157.00000,-6.800000,0.1176,NaN,NaN,LoVoCCS-122
63,123,A3528,193.67000,-29.220000,0.0544,NaN,NaN,LoVoCCS-123
64,131,A761,137.65125,-10.581111,0.0916,NaN,NaN,LoVoCCS-131


## Calculate coverage fraction

In [5]:


# field_xmm_cov_frac = []
# r500_xmm_cov_frac = []
# rvir_xmm_cov_frac = []

# field_chandra_cov_frac = []
# r500_chandra_cov_frac = []
# rvir_chandra_cov_frac = []

for row_ind, row in samp.iterrows():
    cov_calc(row)

KeyError: 'LoVoCCS_name'